In [1]:
import numpy as np
import pandas as pd

In [3]:
#uzeto iz fajla za analizu podataka
kolone = ["ID", "Age", "Gender", "Education", "Country", "Ethnicity", 
            "Nscore", "Escore", "Oscore", "Ascore", "Cscore", "Impulsiveness", "SS", 
            "Alchocol", "Amphet", "Amyl", "Benzos", "Caff", "Canabis", "Choc", "Coke", "Crack", "Ecstasy", 
            "Heroin", "Ketamin", "LegalH", "LSD", "Meth", "Mushrooms", "Nicotine", "Semer", "VSA"]

podaci = pd.read_csv('drug_consumption.data', names = kolone)

podaci = podaci.drop('ID', axis = 1)

In [4]:
klase = []
atributi = []
for x in podaci.columns:
    if x == 'Alchocol':
        break
    atributi.append(x)
    
for x in podaci.columns:
    if x not in atributi:
        klase.append(x)


In [5]:
podaci_skalirano = podaci.copy()

opisni_niz1 = podaci['Alchocol'].unique()
opisni_niz1.sort()
n = len(podaci_skalirano['Heroin'])
    
for x in klase:
    for i in range(n):
        podaci_skalirano[x][i] = int(podaci_skalirano[x][i].strip('CL'))
    print(podaci_skalirano[x].unique())
        

podaci_skalirano

[5 6 4 2 1 0 3]
[2 0 1 3 5 4 6]
[0 2 1 3 5 4 6]
[2 0 3 1 4 5 6]
[6 5 4 3 0 1 2]
[0 4 3 2 1 6 5]
[5 6 4 0 3 2 1]
[0 3 2 1 6 5 4]
[0 1 2 3 5 4 6]
[0 4 1 3 2 6 5]
[0 1 2 3 5 6 4]
[0 2 3 1 5 4 6]
[0 1 2 3 5 4 6]
[0 2 1 3 4 5 6]
[0 3 1 6 4 2 5]
[0 1 2 3 4 5 6]
[2 4 0 6 1 3 5]
[0 2 3 4 1]
[0 1 3 6 2 5 4]


/tmp/ipykernel_6869/4143100454.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  podaci_skalirano[x][i] = int(podaci_skalirano[x][i].strip('CL'))


,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamin,LegalH,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,0,0,0,0,0,0,0,2,0,0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,4,0,2,0,2,3,0,4,0,0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,0,0,0,0,0,0,1,0,0,0
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,0,0,2,0,0,0,0,2,0,0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,1,0,0,1,0,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,...,0,0,0,3,3,0,0,0,0,5
1881,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,...,2,0,0,3,5,4,4,5,0,0
1882,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,...,4,0,2,0,2,0,2,6,0,0
1883,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,...,3,0,0,3,3,0,3,4,0,0


In [9]:
#skaliramo podatke
def skaliraj_podatke(X):
    return (X - min(X))/(max(X) - min(X))

In [10]:
for a in atributi:
    podaci_skalirano[a] = skaliraj_podatke(podaci_skalirano[a])
    
#provera uspesnosti skaliranja
for a in atributi:
    print(len(podaci_skalirano[a].unique()) == len(podaci[a].unique()), end = ", ")
    
#nismo izgubili podatke

True, True, True, True, True, True, True, True, True, True, True, True, 

# pripremam skupove za klasifikaciju

In [34]:
podaci_bez_klasa = podaci_skalirano.copy()
for x in klase:
    podaci_bez_klasa = podaci_bez_klasa.drop(x, axis = 1)
podaci_bez_klasa.head()

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,Impulsiveness,SS
0,0.409137,1.0,0.537681,1.0,0.409061,0.560562,0.412116,0.435690,0.367654,0.499040,0.428474,0.224398
1,0.246475,0.0,1.000000,1.0,0.262143,0.413474,0.796106,0.762567,0.609827,0.479394,0.337920,0.465658
2,0.409137,0.0,0.537681,1.0,0.262143,0.444788,0.622976,0.392939,0.266061,0.353580,0.215401,0.619957
3,0.000000,1.0,0.814329,1.0,0.262143,0.492045,0.376883,0.527023,0.585213,0.584415,0.215401,0.224398
4,0.409137,1.0,1.000000,1.0,0.262143,0.623275,0.250544,0.456995,0.456454,0.688508,0.428474,0.465658


In [123]:
klasa = podaci['Choc']

In [128]:
podaci

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamin,LegalH,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,...,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1881,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,...,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1882,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,...,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1883,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,...,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0


In [129]:
# klasifikacija

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [131]:
X_train, X_test, y_train, y_test = train_test_split(podaci_bez_klasa, klasa, test_size= 0.30, stratify= klasa, random_state= 42)

In [132]:
Bajes = CategoricalNB()
Bajes.fit(X_train, y_train)
pred_trening = Bajes.predict(X_train)
pred_test = Bajes.predict(X_test)

In [133]:
accuracy_score(pred_trening, y_train)

0.4404852160727824

In [134]:
accuracy_score(pred_test, y_test)
#confusion_matrix(y_test, pred)

0.42579505300353354

In [144]:
n = len(klase)
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(podaci_bez_klasa, podaci[klase[i]], test_size= 0.30, stratify= klasa, random_state= 42)
    Bajes.fit(X_train, y_train)
    pred_trening = Bajes.predict(X_train)
    pred_test = Bajes.predict(X_test)
    
    print(klase[i])
    print(f'Trening: {round(accuracy_score(pred_trening, y_train), 2)}')
    print(f'Test: {round(accuracy_score(pred_test, y_test), 2)}')
    print()
    

Alchocol
Trening: 0.41
Test: 0.4

Amphet
Trening: 0.52
Test: 0.52

Amyl
Trening: 0.7
Test: 0.67

Benzos
Trening: 0.52
Test: 0.55

Caff
Trening: 0.73
Test: 0.73

Canabis
Trening: 0.39
Test: 0.36

Choc
Trening: 0.44
Test: 0.43

Coke
Trening: 0.55
Test: 0.55

Crack
Trening: 0.86
Test: 0.87

Ecstasy
Trening: 0.55
Test: 0.54

Heroin
Trening: 0.84
Test: 0.86

Ketamin
Trening: 0.78
Test: 0.81

LegalH
Trening: 0.6
Test: 0.63

LSD
Trening: 0.57
Test: 0.56

Meth
Trening: 0.75
Test: 0.77

Mushrooms
Trening: 0.55
Test: 0.54

Nicotine
Trening: 0.39
Test: 0.34

Semer
Trening: 1.0
Test: 0.99

VSA
Trening: 0.77
Test: 0.77



In [145]:
#deluje da naivni Bajes radi bolje od stabala odlucivanja, proveravam dalje to "tvrdjenje"

In [146]:
podaci_skalirano = podaci.copy()

n = len(podaci_skalirano['Heroin'])
    
for x in klase:
    for i in range(n):
        podaci_skalirano[x][i] = int(podaci_skalirano[x][i].strip('CL'))
    #print(podaci_skalirano[x].unique())
        

podaci_skalirano

/tmp/ipykernel_6869/2160653281.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  podaci_skalirano[x][i] = int(podaci_skalirano[x][i].strip('CL'))


,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamin,LegalH,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,0,0,0,0,0,0,0,2,0,0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,4,0,2,0,2,3,0,4,0,0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,0,0,0,0,0,0,1,0,0,0
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,0,0,2,0,0,0,0,2,0,0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,1,0,0,1,0,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,...,0,0,0,3,3,0,0,0,0,5
1881,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,...,2,0,0,3,5,4,4,5,0,0
1882,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,...,4,0,2,0,2,0,2,6,0,0
1883,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,...,3,0,0,3,3,0,3,4,0,0


In [147]:
for x in klase:
    podaci_skalirano[x] = podaci_skalirano[x].astype('float64')
    
#podaci_skalirano.dtypes

In [151]:
for a in atributi:
    podaci_skalirano[a] = skaliraj_podatke(podaci_skalirano[a])
for a in klase:
    podaci_skalirano[a] = skaliraj_podatke(podaci_skalirano[a])
#podaci_skalirano.describe()

In [152]:
#pravi skupove podataka pojedinacno bez svake klase
podaci_bez_klase = []
for x in klase:
    podaci_bez_klase.append(podaci_skalirano.drop(x, axis = 1))
    
#klasni_podaci, isto to
klasni_podaci = []
n = len(klase)
for i in range(n):
    klasni_podaci.append(podaci[klase[i]])
    
len(podaci_bez_klase) == len(klasni_podaci)

True

In [170]:
n = len(podaci_bez_klase)
for i in range(n):
    
    X_train, X_test, y_train, y_test = train_test_split(podaci_bez_klase[i], podaci[klase[i]], test_size= 0.30, random_state= 42)
    Bajes.fit(X_train, y_train)
    
    pred_trening = Bajes.predict(X_train)    
    print(i)
#     pred_test = Bajes.predict(X_test)
    
    print(klase[i])
    print(f'Trening: {round(accuracy_score(pred_trening, y_train), 2)}')
#     print(f'Test: {round(accuracy_score(pred_test, y_test), 2)}')
    print()

0
Alchocol
Trening: 0.41

1
Amphet
Trening: 0.53

2
Amyl
Trening: 0.69

3
Benzos
Trening: 0.53

4
Caff
Trening: 0.73

5
Canabis
Trening: 0.4

6
Choc
Trening: 0.44

7
Coke
Trening: 0.57

8
Crack
Trening: 0.84

9
Ecstasy
Trening: 0.55

10
Heroin
Trening: 0.84

11
Ketamin
Trening: 0.77

12
LegalH
Trening: 0.61

13
LSD
Trening: 0.59

14
Meth
Trening: 0.76

15
Mushrooms
Trening: 0.56

16
Nicotine
Trening: 0.4

17
Semer
Trening: 0.99

18
VSA
Trening: 0.75

